## Model Predictive Control Example

To determine the optimal control input $u \in \mathbb{R}^m$ applied to the linear time-invariant (LTI) system $\left( A, B\right) \in \left( \mathbb{R}^{n \times n}, \mathbb{R}^{n \times m} \right)$ with state $x \in \mathbb{R}^n$, we solve the following optimization problem:

$
\begin{array}{II}
\text{minimize} \quad & x_H^T P x_H + \sum_{i=0}^{H-1} x_i^T Q x_i + u_i^T R u_i\\
\text{subject to} \quad &x_{i+1} = A x_i + B u_i \quad \forall i \in \{0, ..., H-1\} \\
& \Vert u_i \Vert_\infty \leq 1 \quad \forall i \in \{0, ..., H-1\} \\
& x_0 = x_\mathrm{init} \\
\end{array}
$

The prediction horizon is $H$, the control input is constrained within a box of size $u^\mathrm{max}$ and the cost matrices are positive definite: $P, Q, R \in \mathbb{S}_{++}^n, \mathbb{S}_{++}^n, \mathbb{S}_{++}^m$. Usually, $P$ is chosen as the solution to the discrete-time algebraic Riccati equation for the given LTI system and cost matrices $Q, R$. The initial measured state is $x_\mathrm{init}$. We arrange to state and input variables to matrices $X$ and $U$ with $X_{:,i} = x_i$ and $U_{:,i} = u_i$, respectively, and reformulate the problem to be [DPP-compliant](https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming):

\begin{equation}
\begin{array}{II}
\text{minimize} \quad & \Vert P^{1/2} X_{:,H} \Vert_2^2 + \Vert Q^{1/2} X_{:,0:H-1} \Vert_F^2 + \Vert R^{1/2} U \Vert_F^2\\
\text{subject to} \quad &X_{:,1:H} = A X_{:,0:H-1} + B U \\
& | U | \leq 1 \\
& X_{:,0} = x_\mathrm{init} \\
\end{array}
\end{equation}

Let's define the corresponding CVXPY problem:

In [5]:
import cvxpy as cp

# define dimensions
H, n, m = 10, 3, 2

# define variables
U = cp.Variable((m, H), name='U')
X = cp.Variable((n, H+1), name='X')

# define parameters
Psqrt = cp.Parameter((n, n), name='Psqrt')
Qsqrt = cp.Parameter((n, n), name='Qsqrt')
Rsqrt = cp.Parameter((m, m), name='Rsqrt')
A = cp.Parameter((n, n), name='A')
B = cp.Parameter((n, m), name='B')
x_init = cp.Parameter(n, name='x_init')

# define objective
obj_func = cp.sum_squares(Psqrt@X[:,H-1]) + cp.sum_squares(Qsqrt@X[:,0:H-1]) + cp.sum_squares(Rsqrt@U)

# define constraints
constraints = [X[:,1:] == A@X[:,:H]+B@U,
               cp.abs(U) <= 1,
               X[:,0] == x_init]

# define problem
problem = cp.Problem(objective, constraints)

Assign parameter values and solve the problem.

In [ ]:
# TODO

val = problem.solve()

Generating C source for the problem is as easy as:

In [ ]:
import sys
sys.path.append('../')
import cvxpygen as cpg

cpg.generate_code(problem, code_dir='MPC_code')

Now, you can use a python wrapper around the generated code as a custom CVXPY solve method:

In [ ]:
from MPC_code.cpg_solver import cpg_solve
import numpy as np
import pickle
import time

# load the serialized problem formulation
with open('MPC_code/problem.pickle', 'rb') as f:
    prob = pickle.load(f)

# assign parameter values
# TODO

# solve problem conventionally
t0 = time.time()
# CVXPY chooses eps_abs=eps_rel=1e-5, max_iter=10000, polish=True by default,
# however, we choose the OSQP default values here, as they are used for code generation as well
val = prob.solve(eps_abs=1e-3, eps_rel=1e-3, max_iter=4000, polish=False)
t1 = time.time()
print('\nPython solve time:', 1000*(t1-t0), 'ms')
print('Python objective function value:', val)

# solve problem with C code via python wrapper
prob.register_solve('CPG', cpg_solve)
t0 = time.time()
val = prob.solve(method='CPG')
t1 = time.time()
print('\nC solve time:', 1000*(t1-t0), 'ms')
print('C objective function value:', val)